<center>
<img src="../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 3

### <center> Автор материала: Демо Владимир Олегович

## <center> Индивидуальный проект по анализу данных: </center>
## <center> Прогноз успешного выступления на соревнованиях по Пауэрлифтингу </center>

In [1]:
## подключаемые библиотеки
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing

###  1. Описание набора данных и признаков (2 балла)

Описан процесс сбора данных (если применимо), есть подробное описание решаемой задачи, в чем ее ценность, дано описание целевого и прочих признаков;

__Процесс сбора данных:__

Данные для анализа взяты с Каггл:
https://www.kaggle.com/open-powerlifting/powerlifting-database

Скачать их можно по ссылке:
https://www.kaggle.com/open-powerlifting/powerlifting-database/downloads/powerlifting-database.zip/1

__Подробное описание решаемой задачи:__

Пауэрли́фтинг (англ. powerlifting; power — «сила, мощь» + lifting — «поднятие») или силовое троеборье — силовой вид спорта, суть которого заключается в преодолении сопротивления максимально тяжёлого для спортсмена веса.

Пауэрлифтинг также называют силовым троеборьем. Связано это с тем, что в качестве соревновательных дисциплин в него входят три упражнения (приседания со штангой на спине (точнее на верхней части лопаток), жим штанги лежа на горизонтальной скамье и тяга штанги), которые в сумме определяют квалификацию спортсмена.

На основе данных сайта "Openpowerlifting.org" постараемся дать ответ на следующие вопросы:
    1. Влияет ли категория, в которой выступает пауэрлифтер на результаты соревнований?
    2. Как экипировка влияет на достижение результата?
    3. Можно ли спрогнозировать победу чемпионата на основе первых 2 испытаний (присед и жим)?

__Ценность задачи:__
    У некоторых начинающих спортсменов результаты по отдельным дисциплинам в среднем лучше, чем по другим. Следовательно, необходимо понять, наксколько велик шанс спортсмена достигнуть призового места.

__Описание целевого и прочих признаков:__

Целевой признак:
- Place - место в соревнованиях

Прочие признаки:
- MeetID - ID проведенного мероприятия;
- Name - имя спортсмена (Имя Фамилия на англ. языке);
- Sex - пол спортсмена (male - мужской /female - женский);
- Equipment - экипировка (Multi-ply - многослойная, Raw - без экипировки, Single-ply - однослойная, Straps - бинты на кисти рук, Straps - бинты на колени ног);
- Age - возраст спортсмена в полных годах;
- Division - дивизион в котором выступает спортсмен (возрастная, проверка на допинг и пр.);
- BodyweightKg - вес спортсмена (кг);
- WeightClassKg - весовая категория спортсмена (его вес меньше или равен данной категории, кг);
- BestSquatKg - лучший результат в дисциплине "Присед со штангой" (кг);
- BestBenchKg - лучший результат в дисциплине "Жим штанги лежа" (кг);
- BestDeadliftKg - лучший результат в дисциплине "Становая тяга штанги" (кг);
- TotalKg - сумма трех испытаний (кг);
- Wilks - коэффициент Вилкса, который позволяет на соревнованиях определить абсолютного чемпионата вне зависимости от весовой категории.

In [4]:
op = pd.read_csv('../../data/openpowerlifting.csv')
op.shape

(386414, 17)

###  2. Первичный анализ признаков

Исследованы признаки, их взаимодействия, влияние на целевой признак. Исследовано распределение целевого признака (в случае задачи регрессии проведены стат-тесты на нормальность и скошенность (skewness) распределения). Если необходимо, объясняется, почему и как можно преобразовать целевой признак. Изучены выбросы и пропуски в данных;

In [3]:
op.head()

,MeetID,Name,Sex,Equipment,Age,Division,BodyweightKg,WeightClassKg,Squat4Kg,BestSquatKg,Bench4Kg,BestBenchKg,Deadlift4Kg,BestDeadliftKg,TotalKg,Place,Wilks
0,0,Angie Belk Terry,F,Wraps,47.0,Mst 45-49,59.60,60,NaN,47.63,NaN,20.41,NaN,70.31,138.35,1,155.05
1,0,Dawn Bogart,F,Single-ply,42.0,Mst 40-44,58.51,60,NaN,142.88,NaN,95.25,NaN,163.29,401.42,1,456.38
2,0,Dawn Bogart,F,Single-ply,42.0,Open Senior,58.51,60,NaN,142.88,NaN,95.25,NaN,163.29,401.42,1,456.38
3,0,Dawn Bogart,F,Raw,42.0,Open Senior,58.51,60,NaN,NaN,NaN,95.25,NaN,NaN,95.25,1,108.29
4,0,Destiny Dula,F,Raw,18.0,Teen 18-19,63.68,67.5,NaN,NaN,NaN,31.75,NaN,90.72,122.47,1,130.47


In [4]:
print(op.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386414 entries, 0 to 386413
Data columns (total 17 columns):
MeetID            386414 non-null int64
Name              386414 non-null object
Sex               386414 non-null object
Equipment         386414 non-null object
Age               147147 non-null float64
Division          370571 non-null object
BodyweightKg      384012 non-null float64
WeightClassKg     382602 non-null object
Squat4Kg          1243 non-null float64
BestSquatKg       298071 non-null float64
Bench4Kg          1962 non-null float64
BestBenchKg       356364 non-null float64
Deadlift4Kg       2800 non-null float64
BestDeadliftKg    317847 non-null float64
TotalKg           363237 non-null float64
Place             385322 non-null object
Wilks             362194 non-null float64
dtypes: float64(10), int64(1), object(6)
memory usage: 50.1+ MB
None


### 7. Предобработка данных (4 балла)

Проведена предобработка данных для конкретной модели. При необходимости есть и описано масштабирование признаков, заполнение пропусков, замены строк на числа, OheHotEncoding, обработка выбросов, отбор признаков с описанием используемых для этого методов. Корректно сделано разбиение данных на обучающую и отложенную части;

Проверим данные на наличие пропусков. Определим долю пропусков от общего числа данных

In [5]:
op.isnull().sum() / op.shape[0]

MeetID            0.000000
Name              0.000000
Sex               0.000000
Equipment         0.000000
Age               0.619199
Division          0.041000
BodyweightKg      0.006216
WeightClassKg     0.009865
Squat4Kg          0.996783
BestSquatKg       0.228623
Bench4Kg          0.994923
BestBenchKg       0.077766
Deadlift4Kg       0.992754
BestDeadliftKg    0.177444
TotalKg           0.059980
Place             0.002826
Wilks             0.062679
dtype: float64

Столцы Squat4Kg, Bench4Kg, Deadlift4Kg в большинстве отсутствуют (>99%) и age в 62% случаев, следовательно данные признаки удаляем из dataset

In [6]:
op=op.drop(["Squat4Kg", "Bench4Kg", "Deadlift4Kg", "Age"], axis=1)

In [7]:
op.describe()

,MeetID,BodyweightKg,BestSquatKg,BestBenchKg,BestDeadliftKg,TotalKg,Wilks
count,386414.000000,384012.000000,298071.000000,356364.000000,317847.000000,363237.000000,362194.000000
mean,5143.015804,86.934912,176.569941,118.347509,195.040633,424.000249,301.080601
std,2552.099838,23.140843,69.222785,54.848850,61.580675,196.355147,116.360396
min,0.000000,15.880000,-477.500000,-522.500000,-410.000000,11.000000,13.730000
25%,2979.000000,70.300000,127.500000,79.380000,147.500000,272.160000,237.380000
50%,5960.000000,83.200000,174.630000,115.000000,195.000000,424.110000,319.660000
75%,7175.000000,100.000000,217.720000,150.000000,238.140000,565.000000,379.290000
max,8481.000000,242.400000,573.790000,488.500000,460.400000,1365.310000,779.380000


Также имеем в столбцах BestSquatKg,	BestBenchKg, BestDeadliftKg отрицательные значения, что по физической природе является выбросом. Спишем это на ошибки ввода, следовательно данные параметры необходимо взять по модулю

In [8]:
op['BestSquatKg']=np.abs(op['BestSquatKg'])
op['BestBenchKg']=np.abs(op['BestBenchKg'])
op['BestDeadliftKg']=np.abs(op['BestDeadliftKg'])

Заменим все NaN в столбцах BestSquatKg, BestBenchKg, BestDeadliftKg, TotalKg, Wilks на "нули" (спортсмену не удалось выполнить начальный вес в состязаниях, поэтому у него суммарный вес и Вилкс тоже "ноль"

In [9]:
op['BestSquatKg']=op['BestSquatKg'].fillna(0)
op['BestBenchKg']=op['BestBenchKg'].fillna(0)
op['BestDeadliftKg']=op['BestDeadliftKg'].fillna(0)
op['TotalKg']=op['TotalKg'].fillna(0)
op['Wilks']=op['Wilks'].fillna(0)

Проверим также на корректность итоговую сумму трех дисциплин. Создадим отдельную колонку, где покажем процент ошибки ввода данных. Затем удалим строчки, где ошибка больше 1%

In [10]:
op['TotalKg_check']=(op['TotalKg']-(op['BestSquatKg']+op['BestBenchKg']+op['BestDeadliftKg']))/op['TotalKg']
op=op.drop(op[np.abs(op['TotalKg_check'])>0.01].index)

op=op.drop(['TotalKg_check'], axis=1)
op.describe()

,MeetID,BodyweightKg,BestSquatKg,BestBenchKg,BestDeadliftKg,TotalKg,Wilks
count,372576.000000,370219.000000,372576.000000,372576.000000,372576.000000,372576.000000,372576.000000
mean,5119.195866,86.840444,137.287194,111.898557,164.003853,413.189168,292.569356
std,2561.807728,23.102765,94.101900,57.697462,90.281214,205.010422,125.076258
min,0.000000,15.880000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2895.000000,70.200000,75.000000,70.000000,120.000000,260.000000,223.530000
50%,5947.000000,83.000000,147.500000,112.500000,180.000000,415.040000,316.090000
75%,7166.000000,100.000000,204.120000,147.500000,230.000000,562.400000,377.330000
max,8481.000000,242.400000,573.790000,488.500000,460.400000,1365.310000,779.380000


Заметим, что еще остались значения Nan в признаке "BodyweightKg", следовательно удалим все строчки, где данный признак не указан, чтобы количество элементов в каждом признаке было одинаковым.

In [11]:
op=op.drop(op[op["BodyweightKg"].isnull()].index)
op.describe()

,MeetID,BodyweightKg,BestSquatKg,BestBenchKg,BestDeadliftKg,TotalKg,Wilks
count,370219.000000,370219.000000,370219.000000,370219.000000,370219.000000,370219.000000,370219.000000
mean,5115.469549,86.840444,137.755888,112.177891,164.609692,414.543030,294.432000
std,2565.110974,23.102765,93.731000,57.264106,89.787171,203.550292,123.268996
min,0.000000,15.880000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2869.000000,70.200000,75.000000,70.310000,120.000000,260.800000,226.830000
50%,5944.000000,83.000000,147.500000,112.500000,180.000000,417.300000,316.910000
75%,7169.000000,100.000000,204.120000,147.500000,230.000000,562.450000,377.740000
max,8481.000000,242.400000,573.790000,488.500000,460.400000,1365.310000,779.380000


Получили "очищенные данные", которые демонстрируют статистику по итоговым места в соревнованиях из 370219 строчек

In [12]:
op.groupby('Place')
place_stat=op['Place'].value_counts()
place_stat[0:10]

1     194006
2      57024
3      29938
4      19038
5      13431
6       9961
DQ      8113
7       7775
8       6063
9       4746
Name: Place, dtype: int64

In [13]:
op['Place'].hist(bins=25)

"Место" в соревнованиях определим как целевой признак. Чтобы в дальнейшем можно было провести прогноз, необходимо выполнить еще несколько подготовительных пунктов:
- создание новых признаков
- перевести текстовые значения в числовые для возможностей анализа и прогноза

### 9. Создание новых признаков и описание этого процесса (4 балла)

Созданы новые признаки. Дано обоснование: логическое (например, у птиц температура тела на несколько градусов выше человеческой, значит вирус ХХХ не выживет в такой среде), физическое (например, радуга означает, что источник света расположен сзади; расчет величины по физическому закону с использованием данных признаков) или другое (скажем, признак построен после визуализации данных). Обоснование разумно описано. Полезность новых признаков подтверждена статистически или с помощью соответствующей модели;

Создаем новый признак: "Число участий в соревнований" - 'MeetCount'. Каждый спортсмен будет иметь данные по сумме соревнований, что в дальнейшем позволяет понять насколько он опытный.

In [14]:
op2=op.drop(['Sex','Equipment','Division','BodyweightKg','WeightClassKg','BestSquatKg','BestBenchKg','BestDeadliftKg','TotalKg','Wilks','Place'], axis=1)
op3=op2.groupby('Name').agg(np.count_nonzero)
op4=op3.rename(columns={'MeetID': 'MeetCount'})
op4['Name']=op4.index
op = op.merge(op4, 'left', on='Name')
op.head()

,MeetID,Name,Sex,Equipment,Division,BodyweightKg,WeightClassKg,BestSquatKg,BestBenchKg,BestDeadliftKg,TotalKg,Place,Wilks,MeetCount
0,0,Angie Belk Terry,F,Wraps,Mst 45-49,59.60,60,47.63,20.41,70.31,138.35,1,155.05,0
1,0,Dawn Bogart,F,Single-ply,Mst 40-44,58.51,60,142.88,95.25,163.29,401.42,1,456.38,9
2,0,Dawn Bogart,F,Single-ply,Open Senior,58.51,60,142.88,95.25,163.29,401.42,1,456.38,9
3,0,Dawn Bogart,F,Raw,Open Senior,58.51,60,0.00,95.25,0.00,95.25,1,108.29,9
4,0,Destiny Dula,F,Raw,Teen 18-19,63.68,67.5,0.00,31.75,90.72,122.47,1,130.47,0


В дальнейшем признак 'Name' нам не понадобится, поэтому его удаляем.

In [15]:
op=op.drop(['Name'], axis=1)

Переводим категориальные признаки в числовые

In [16]:
op['Division'] = pd.factorize(op.Division)[0]
op['Equipment'] = pd.factorize(op.Equipment)[0]
op['Sex'] = pd.factorize(op.Sex)[0]
op['WeightClassKg'] = pd.factorize(op.Place)[0]
op['Place'] = pd.factorize(op.Place)[0]

In [17]:
op.head()

,MeetID,Sex,Equipment,Division,BodyweightKg,WeightClassKg,BestSquatKg,BestBenchKg,BestDeadliftKg,TotalKg,Place,Wilks,MeetCount
0,0,0,0,0,59.60,0,47.63,20.41,70.31,138.35,0,155.05,0
1,0,0,1,1,58.51,0,142.88,95.25,163.29,401.42,0,456.38,9
2,0,0,1,2,58.51,0,142.88,95.25,163.29,401.42,0,456.38,9
3,0,0,2,2,58.51,0,0.00,95.25,0.00,95.25,0,108.29,9
4,0,0,2,3,63.68,0,0.00,31.75,90.72,122.47,0,130.47,0


Признак "place" имеет признак: "место" в числовом значении и "дисквалификация" в буквенном. Нас не интересует не 1 место, следовательно "place" который не равен 1 (после факторризации 0) будет равен 0, а остальное 1

In [18]:
op['Place'][op['Place']==0]=0
op['Place'][op['Place']!=0]=1

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [19]:
op.head()

,MeetID,Sex,Equipment,Division,BodyweightKg,WeightClassKg,BestSquatKg,BestBenchKg,BestDeadliftKg,TotalKg,Place,Wilks,MeetCount
0,0,0,0,0,59.60,0,47.63,20.41,70.31,138.35,0,155.05,0
1,0,0,1,1,58.51,0,142.88,95.25,163.29,401.42,0,456.38,9
2,0,0,1,2,58.51,0,142.88,95.25,163.29,401.42,0,456.38,9
3,0,0,2,2,58.51,0,0.00,95.25,0.00,95.25,0,108.29,9
4,0,0,2,3,63.68,0,0.00,31.75,90.72,122.47,0,130.47,0


### 3. Первичный визуальный анализ данных (4 балла)

Построены визуализации (распределения признаков, матрица корреляций и т.д.), описана связь с анализом данным (п. 2). Присутствуют выводы;

In [20]:
sns.heatmap(op.corr())

In [21]:
corr = op.corr()
corr

,MeetID,Sex,Equipment,Division,BodyweightKg,WeightClassKg,BestSquatKg,BestBenchKg,BestDeadliftKg,TotalKg,Place,Wilks,MeetCount
MeetID,1.000000,-0.003175,0.082804,0.388125,-0.003192,0.083822,0.009436,-0.082722,0.047075,0.001837,0.063686,0.004191,-0.129235
Sex,-0.003175,1.000000,-0.105358,0.063247,0.411728,0.085944,0.256889,0.477586,0.270738,0.372074,0.107884,0.042822,0.057505
Equipment,0.082804,-0.105358,1.000000,-0.010413,-0.036853,-0.031549,-0.211148,-0.054256,-0.099984,-0.156596,-0.042622,-0.141921,-0.062792
Division,0.388125,0.063247,-0.010413,1.000000,-0.067005,0.216023,-0.004188,-0.161636,-0.035943,-0.063255,0.152912,-0.064365,-0.089952
BodyweightKg,-0.003192,0.411728,-0.036853,-0.067005,1.000000,-0.045611,0.249665,0.521375,0.248469,0.371244,-0.033973,0.018614,0.063236
WeightClassKg,0.083822,0.085944,-0.031549,0.216023,-0.045611,1.000000,0.107688,-0.066430,0.051458,0.053598,0.549941,0.054178,-0.105522
BestSquatKg,0.009436,0.256889,-0.211148,-0.004188,0.249665,0.107688,1.000000,0.450470,0.800046,0.940114,0.117185,0.889166,-0.063222
BestBenchKg,-0.082722,0.477586,-0.054256,-0.161636,0.521375,-0.066430,0.450470,1.000000,0.292183,0.617642,-0.095160,0.404976,0.152302
BestDeadliftKg,0.047075,0.270738,-0.099984,-0.035943,0.248469,0.051458,0.800046,0.292183,1.000000,0.891709,0.039465,0.832557,-0.095635
TotalKg,0.001837,0.372074,-0.156596,-0.063255,0.371244,0.053598,0.940114,0.617642,0.891709,1.000000,0.044599,0.890619,-0.028452


### 4. Инсайты, найденные зависимости (4 балла)

Найдены и выдвинуты предположения о природе различных корреляций/пропусков/закономерностей и выбросов, найденных в предыдущих пунктах. Есть пояснение, почему они важны для решаемой задачи;

Насколько видим из матрицы корреляции, место ('Place' - целевой признак) коррелирует лучше всего с WeightClassKg (весовая категория, в которой выступает спортсмен). Чем выше категория, тем меньше вероятность победы.

С экипировкой никакой корреляции нет.

Для прогнозы победы на основе двух первых испытаний (присед и жим) необходимо удалить данные по становой тяге, суммарный взятый вес и Вилкс, так как они являются результатом соревнований и напрямую прогнозируют победу.

In [42]:
op=op.drop(['BestDeadliftKg', 'TotalKg','Wilks'], axis=1)
op.head()

,MeetID,Sex,Equipment,Division,BodyweightKg,WeightClassKg,BestSquatKg,BestBenchKg,Place,MeetCount
0,0,0,0,0,59.60,0,47.63,20.41,0,0
1,0,0,1,1,58.51,0,142.88,95.25,0,9
2,0,0,1,2,58.51,0,142.88,95.25,0,9
3,0,0,2,2,58.51,0,0.00,95.25,0,9
4,0,0,2,3,63.68,0,0.00,31.75,0,0


### 5. Выбор метрики (3 балла)

Есть разумное обоснование выбора метрики качества модели. Описаны моменты, влияющие на выбор метрики качества (решаемая задача, цель решения, количество классов, дисбаланс классов, прочее);

In [53]:
target = op['Place']
train = op.drop(['Place'], axis=1) 

In [54]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.cross_validation import cross_val_score

In [55]:
dt = DecisionTreeClassifier(random_state=17, class_weight='balanced')

In [56]:
max_depth_values = range(1, 10)
max_features_values = range(1, 10)
tree_params = {'max_depth': max_depth_values,
               'max_features': max_features_values}

In [57]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)
from sklearn.metrics import accuracy_score
X = train
y = target

In [58]:
grid=GridSearchCV(dt,tree_params,cv=skf, n_jobs=-1,scoring='roc_auc',verbose=1)

In [59]:
grid.fit(X, y)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   46.6s
[Parallel(n_jobs=-1)]: Done 405 out of 405 | elapsed:  2.0min finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=17, shuffle=True),
       error_score='raise',
       estimator=DecisionTreeClassifier(class_weight='balanced', criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=17,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': range(1, 10), 'max_features': range(1, 10)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=1)

In [61]:
from sklearn.ensemble import RandomForestClassifier

In [62]:
dt2 = RandomForestClassifier(n_jobs=1, random_state=17)

In [63]:
max_depth_values = range(1, 10)
max_features_values = range(1, 10)
tree_params = {'max_depth': max_depth_values,
               'max_features': max_features_values}

In [64]:
grid2=GridSearchCV(dt2,tree_params,cv=skf, n_jobs=-1,scoring='roc_auc',verbose=1)

In [65]:
grid2.fit(X, y)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   28.0s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 405 out of 405 | elapsed:  6.2min finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=17, shuffle=True),
       error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=17, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': range(1, 10), 'max_features': range(1, 10)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=1)

### 8. Кросс-валидация и настройка гиперпараметров модели (4 балла)

Кросс-валидация выполнена технически верно, нет утечек данных. Разумно выбрано количество фолдов и разбиение (Random/Stratified или иное), зафиксирован seed. Присутствует объяснение. Объяснены гиперпараметры модели и способ их выбора. Выбор основан на некотором исследовании гипрепараметров модели для данной задачи;

In [67]:
clf_best_score = grid.best_score_
clf_best_params = grid.best_params_
clf_best = grid.best_estimator_
mean_validation_scores = []
print("Лучший результат в дереве решений", clf_best_score, 
      "лучшие параметры", clf_best_params)

Лучший результат в дереве решений 1.0 лучшие параметры {'max_depth': 2, 'max_features': 5}


In [68]:
clf_best_score2 = grid2.best_score_
clf_best_params2 = grid2.best_params_
clf_best2 = grid2.best_estimator_
print("Лучший результат в случайном лесе", clf_best_score2, 
      "лучшие параметры", clf_best_params2)

Лучший результат в случайном лесе 1.0 лучшие параметры {'max_depth': 2, 'max_features': 6}


### 6. Выбор модели (3 балла)

Произведен выбор модели. Описан процесс выбора и связь с решаемой задачей;

Как видно, "случайный лес" и "дерево решений" дают точный прогноз (100%) победы спортсмена

### 11. Прогноз для тестовой или отложенной выборке (2 балла)

Указаны результаты на тестовой выборке или LB score. Результаты на тестовой выборке сравнимы с результатами на кросс-валидации. Если тестовая выборка создавалась автором проекта, то механизм создания должен быть непредвзят и объяснен (применен разумный механизм выборки, в простейшем случае – рандомизация);

Выполним прогноз при помощи "случайного леса" с лучшими параметрами на тренировочной и отложенной выборке

In [69]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(train,target,test_size=0.3, random_state=17)

In [70]:
dt3 = RandomForestClassifier(n_jobs=1, max_depth=2, max_features=6,random_state=17)
dt3.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features=6, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=17, verbose=0, warm_start=False)

In [71]:
dt3_pred = dt3.predict(X_valid)
accuracy_score(y_valid, dt3_pred)

0.99725388507734136

In [72]:
dt4 = DecisionTreeClassifier(class_weight='balanced',max_depth=2, max_features=5,random_state=17)
dt4.fit(X_train, y_train)

DecisionTreeClassifier(class_weight='balanced', criterion='gini', max_depth=2,
            max_features=5, max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=17, splitter='best')

In [73]:
dt4_pred = dt4.predict(X_valid)
accuracy_score(y_valid, dt4_pred)

1.0

В итоге, "дерево решений" отлично справляется с прогнозом победы на соревнованиях по Пауэрлифтингу по результатам отложенной выборки (100%), в отличие от "случайного леса", где прогноз хуже (99.7%)

### 12. Выводы (2 балла)

Описана ценность решения, возможности применения, дальнейшие пути развития и улучшения решения;

Прогноз победы на соревнованиях по Пауэрлифтингу имеет низкую ценность, так как на такого рода соревнованиях отсутствует тотализатор. При этом на основе первичных данных о спортсмене, его опыт, экипировка, весовая категория и результаты по первым двум испытаниям есть возможность спрогнозировать победу со 100% вероятностью.